In [1]:
import math
import numpy as np
from collections import namedtuple
from decimal import *

from typing import List, Tuple
from enum import Enum

import xlrd

# 変更履歴

- 2017.7: 玄関の土間床を追加

# 四捨五入

建研HPで公表する値としては、さらに小数点2桁までにまとめた値とする。  
丸める場合に、round関数は、JIS Z 8401 付属書A の方法（0.5の場合は偶数側に丸める）が適用されるため、自前に作成したround2関数で丸めることとする。この場合は、端数が「.5」の場合は、必ず大きい側（切り上げ側）に丸められる。

In [2]:
def round2(x,d=2):
    p=10**d
    return (x*p*2+1)//2/p
#    return x.quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)

# 住宅の種類

床・基礎部分の断熱方法

In [3]:
class FloorIns(Enum):

    # 床断熱
    FLOOR = 1
    
    # 基礎断熱
    BASE = 2

# 保持するパラメータ

In [4]:
class House():
    
    def __init__(self, name, ver, floor_ins):
        
        # 名前
        self.name = name
        
        # バージョン
        self.ver = ver

        # 床の断熱種類
        self.floor_ins = floor_ins
        
        # 床面積の合計, m2
        self.a_f_total = None
        
        # 形状係数
        self.sf = None
        
        # 1階の床面積に対する2階の床面積の比
        self.r_fa = None
        
        # 基礎の高さ（一般部分）, m
        self.h_base = None
        
        # 基礎の高さ（玄関）, m
        self.h_base_entrance = None
        
        # 基礎の高さ（浴室）, m
        self.h_base_bath = None
        
        # 高さ, m
        self.h = None
        
        # ドアの面積（玄関）, m2
        self.a_door_entrance = None
        
        # ドアの面積（勝手口）, m2
        self.a_door_backentrance = None
        
        # 玄関の土間床等の周辺部の基礎長さ（主開口方位側）, m
        self.l_base_entrance_ms = None
        
        # 玄関の土間床等の周辺部の基礎長さ（その他方位側）, m
        self.l_base_entrance_os = None
        
        # 浴室の土間床等の周辺部の基礎長さ（主開口方位側）, m
        self.l_base_bath_ms = None
        
        # 浴室の土間床等の周辺部の基礎長さ（その他方位側）, m
        self.l_base_bath_os = None
        
        # 開口部の面積比率
        self.r_opn = None
        
        # 方位別窓面積比率
        self.r_wnd = None
        
        # 床面積（土間床含む）, m2        
        self.a_f = None
        
        # 周長, m
        self.l_prm = None

        # 長辺の長さ, m
        self.l_ls = None
        
        # 短辺の長さ, m
        self.l_ss = None
        
        # 玄関の土間床等の面積, m2
        self.a_ef_entrance = None
        
        # 浴室の土間床等の面積, m2
        self.a_ef_bath = None
        
        # その他の土間床等の面積, m2
        self.a_ef_other = None
        
        # 床断熱した床の面積, m2
        self.a_if = None
        
        # 屋根又は天井の面積, m2
        self.a_roof = None
        
        # 玄関等の土間床等の外周部の長さ（外気側）, m
        self.l_base_entrance_outside = None
        
        # 玄関等の土間床等の外周部の長さ（床下側）, m
        self.l_base_entrance_inside = None
        
        # 浴室の土間床等の外周部の長さ（外気側）, m
        self.l_base_bath_outside = None
        
        # 浴室の土間床等の外周部の長さ（床下側）, m
        self.l_base_bath_outside = None
        
        # その他の土間床等の外周部の長さ（外気側）, m
        self.l_base_other_outside = None
        
        # その他の土間床等の外周部の長さ（床下側）, m
        self.l_base_other_inside = None
        
        # 土間床周長（外気側）, m
        self.l_prm_outside = None
        
        # 土間床周長（床下側）, m
        self.l_prm_inside = None
        
        # 玄関等の基礎の面積（外気側）, m2
        self.a_base_entrance_outside = None
        
        # 玄関等の基礎の面積（床下側）, m2
        self.a_base_entrance_inside = None
        
        # 浴室の基礎の面積（外気側）, m2
        self.a_base_bath_outside = None
        
        # 浴室の基礎の面積（床下側）, m2
        self.a_base_bath_inside = None
        
        # その他の基礎の面積（外気側）, m2
        self.a_base_other_outside = None
        
        # その他の基礎の面積（床下側）, m2
        self.a_base_other_inside = None
        
        # 基礎の面積の合計（外気側）, m2
        self.a_base_total_outside = None
        
        # 基礎の面積の合計（床下側）, m2
        self.a_base_total_inside = None
        
        # 垂直外皮面積, m2
        self.a_srf = None
        
        # 基礎を除く外皮の部位の面積の合計, m2
        self.a_dash_env = None
        
        # 外皮の部位の面積の合計, m2
        self.a_env = None
        
        # 開口部の面積の合計, m2
        self.a_opn = None
        
        # 窓の面積の合計, m2
        self.a_wnd_total = None

        # 窓の面積, m2
        self.a_wnd = None
        
        # ドアの面積, m2
        self.a_door = None

        # 壁の面積, m2
        self.a_wall = None

# 計算ケース

以下のケースについて検討する。
- 床断熱住宅（外皮に接する浴室床なし）
- 床断熱住宅（浴室床断熱）
- 床断熱住宅（浴室基礎断熱）
- 基礎断熱住宅

In [5]:
hs = [
    House(name='床断熱住宅', ver=1, floor_ins=FloorIns.FLOOR),
    House(name='基礎断熱住宅', ver=1, floor_ins=FloorIns.BASE)
]

# 公表資料の値

In [6]:
docs = [
    {},
    {}
]

# 簡易評価式の算出の前提

部位の面積及び長さ等を計算するにあたっての前提条件を以下に示す。

| 意味 | 記号| 値 | 単位 | 備考 |
| --- | :---: | :---: | :---: | --- |
| 階数 | - | 2 | - | - |
| 床面積の合計 | $A_{F,total}$ | 90.0 | m<sup>2</sup> | - |
| 形状係数(1階) | $SF_1$ | 1.08 | - | - |
| 形状係数(2階) | $SF_2$ | 1.04 | - | - |
| 主開口方位 | - | 南西 | - | - |
| 1階の床面積に対する2階の床面積の比 | $r_{FA}$ | 0.77 | - | 自立循環型住宅プランの比を採用した。 |
| 基礎の高さ(基礎断熱住宅) | $h_{base}$ | 0.5 | m | 基礎の高さとは土間基礎上端から1階床レベルまでの間を言う。<br>(GLからではない。) |
| 基礎の高さ(玄関) | $h_{base,d}$ | 0.18 | m | - |
| 基礎の高さ(浴室) | $h_{base,b}$ | 0.5 | m | - |
| 高さ(1階) | $h_1$ | 2.9 | m | - |
| 高さ(2階) | $h_2$ | 2.7 | m | - |
| ドア(玄関) | $A_{door,1}$ | 1.89 | m<sup>2</sup> | 方位：北西 自立循環型住宅プランの面積を採用 |
| ドア(勝手口) | $A_{door,2}$ | 1.62 | m<sup>2</sup> | 方位：北東 自立循環型住宅プランの面積を採用 |
| 玄関の土間床等の周辺部の基礎長さ(主開口方位側) | $L_{base,d,MS}$ | 1.365 | m | - |
| 玄関の土間床等の周辺部の基礎長さ(その他の方位側) | $L_{base,d,OS}$ | 1.82 | m | - |
| 浴室の土間床等の周辺部の基礎長さ(主開口方位側) | $L_{base,b,MS}$ | 1.82 | m | - |
| 浴室の土間床等の周辺部の基礎長さ(その他の方位側) | $L_{base,b,OS}$ | 1.82 | m | - |
| 開口部の面積比率 | $r_{opn}$ | 0.14 | - | 床断熱住宅と基礎断熱住宅とを同じ開口部面積にするために、<br>基礎面積を含めない外皮表面積に対する値とする。 |
| 方位別窓面積比率(南西) | $r_{wnd,SW}$ | 0.6862 | - | - |
| 方位別窓面積比率(北西) | $r_{wnd,NW}$ | 0.0721 | - | - |
| 方位別窓面積比率(北東) | $r_{wnd,NE}$ | 0.1097 | - | - |
| 方位別窓面積比率(南東) | $r_{wnd,SE}$ | 0.1320 | - | - |


※基礎の高さについて  
下記を想定すると、根太がけの場合は524～539、根太レスの場合は479～494  
・基礎の高さは400mm（参考：長期優良住宅）  
・土台は105mm又は10mm  
・根太は45mm（根太がけの場合のみ）  
・床荒板およびフローリングは24mm（12mm＋12mm）  
・GLから土間基礎上端までは50mm  
あるいは下記を想定すると、根太がけの場合は489～504、根太レスの場合は444～459  
・床下空間高さ（土間基礎上端から大引き下端まで）は330mmを確保（長期優良）  
・大引は90又は105mm  
・根太は45mm（根太がけの場合のみ）  
・床荒板およびフローリングは24mm（12mm＋12mm）  
以上を勘案して、500mmと決めた。

## 階数

階数は2階とする。

##  床面積の合計

$ A_{F,total} = 90.0 $ (m<sup>2</sup>)

In [7]:
for h in hs:
    h.a_f_total = 90.0

## 形状係数

1階: $ SF_1 = 1.08 $  
2階: $ SF_2 = 1.04 $

In [8]:
for h in hs:
    h.sf = [1.08, 1.04]

## 主開口方位

南西とする。

## 1階の床面積に対する2階の床面積の比

自立循環型住宅プランの比を採用する。

$ r_{FA} = 0.77 $

In [9]:
for h in hs:
    h.r_fa = 0.77

## 基礎の高さ

$ h_{base} = 0.5 $ (m) (一般部分)  

$ h_{base,entrance} = 0.18 $ (m) (玄関)

$ h_{base,bath} = 0.5 $ (m) (浴室)

In [10]:
for h in hs:
    h.h_base = 0.5
    h.h_base_entrance = 0.18
    h.h_base_bath = 0.5

## 高さ

$ h_1 = 2.9 $ (1階)  
$ h_2 = 2.7 $ (2階)

In [11]:
for h in hs:
    h.h = [2.9, 2.7]

## ドアの面積

玄関の方位は北西、勝手口の方位は北東とする。自立循環型住宅プランの面積を採用した。

$ A_{door,entrance} = 1.89 $ (m<sup>2</sup>) (玄関)  

$ A_{door,backentrance} = 1.62 $ (m<sup>2</sup>) (勝手口)

In [12]:
for h in hs:
    h.a_door_entrance = 1.89
    h.a_door_backentrance = 1.62

## 玄関の土間床等の周辺部の基礎長さ

$L_{base,entrance,MS}=1.365$(m)(主開口方位側)

$L_{base,entrance,OS}=1.82$(m)(その他方位側)

In [13]:
for h in hs:
    h.l_base_entrance_ms = 1.365
    h.l_base_entrance_os = 1.82

## 浴室の土間床等の周辺部の基礎長さ

$L_{base,bath,MS}=1.82$(m)(主開口方位側)

$L_{base,bath,OS}=1.82$(m)(その他方位側)

In [14]:
for h in hs:
    h.l_base_bath_ms = 1.82
    h.l_base_bath_os = 1.82

## 開口部の面積比率

$r_{opn} = 0.12$

床断熱住宅と基礎断熱住宅とを同じ開口部面積にするために、基礎面積を含めない外皮表面積に対する値とする。

In [15]:
for h in hs:
    h.r_opn = 0.14

## 方位別窓面積比率

南西  
$r_{wnd,SW}=0.6862$  
北西  
$r_{wnd,NW}=0.0721$  
北東  
$r_{wnd,NE}=0.1097$  
南東  
$r_{wnd,SE}=0.1320$

In [16]:
for h in hs:
    h.r_wnd = [0.6862, 0.0721, 0.1097, 0.1320]

# 算出過程

In [17]:
def show_result(prm):
    for h in hs:
        print(str(h.name) + ' : ' + str(prm(h)))

## 床面積(土間床含む)

$ A_{F,1} = \frac{A_{F,total}}{ ( 1 + r_{FA} )} $  

$ A_{F,2} = A_{F,total} - A_{F,1} $

$A_{F,total}$: 床面積の合計(m<sup>2</sup>)  
$r_{FA}$: 1階の床面積に対する2階の床面積の比  
$A_{F,1}$: 1階の床面積(土間床含む)(m<sup>2</sup>)  
$A_{F,2}$: 2階の床面積(m<sup>2</sup>)

In [18]:
for h in hs:
    a_f_1 = h.a_f_total / (1.0 + h.r_fa)
    a_f_2 = h.a_f_total - a_f_1
    h.a_f = [a_f_1, a_f_2]

In [19]:
show_result(lambda h: h.a_f)

床断熱住宅 : [50.847457627118644, 39.152542372881356]
基礎断熱住宅 : [50.847457627118644, 39.152542372881356]


## 周長

$ \begin{align} L_{prm,i} = \sqrt{A_{F,i}} \times 4 \times SF_i \end{align} $

$L_{prm,i}$: $i$階の周長(m)($i=1,2$)  
$SF_i$: $i$階の形状係数($i=1,2$)  

In [20]:
for h in hs:
    h.l_prm = [4 * sf_i * np.sqrt(a_f_i) for a_f_i, sf_i in zip(h.a_f, h.sf)]

In [21]:
show_result(lambda h: h.l_prm)

床断熱住宅 : [30.80479821749104, 26.029948852968104]
基礎断熱住宅 : [30.80479821749104, 26.029948852968104]


## 長辺の長さ

$ \displaystyle L_{LS,i} = \frac{ \frac{L_{prm,i}}{2} + \sqrt{ \left( \frac{L_{prm,i}}{2} \right)^2 - 4 \times A_{F,i} } }{2} $

$L_{LS,i}$: $i$階の長辺の長さ($i=1,2$)

In [22]:
for h in hs:
    h.l_ls = [(l_prm_i / 2 + np.sqrt((l_prm_i / 2)**2 - 4 * a_f_i)) / 2
              for l_prm_i, a_f_i in zip(h.l_prm, h.a_f)]

In [23]:
show_result(lambda h: h.l_ls)

床断熱住宅 : [10.609982280729707, 8.29490083566609]
基礎断熱住宅 : [10.609982280729707, 8.29490083566609]


## 短辺の長さ

$ \displaystyle L_{SS,i} = \frac{L_{prm,i}}{2} - L_{LS,i} $

$L_{ss,i}$: $i$階の短辺の長さ(m)

In [24]:
for h in hs:
    h.l_ss = [l_prm_i / 2 - l_ls_i for l_prm_i, l_ls_i in zip(h.l_prm, h.l_ls)]

In [25]:
show_result(lambda h: h.l_ss)

床断熱住宅 : [4.7924168280158135, 4.720073590817963]
基礎断熱住宅 : [4.7924168280158135, 4.720073590817963]


## 玄関の土間床等の面積

$ A_{EF,entrance} = L_{base,entrance,MS} \times L_{base,entrance,OS} $

$A_{EF,entrance}$: 玄関の土間床等の面積(m<sup>2</sup>)  
$L_{base,entrance,MS}$: 玄関の土間床等の周辺部の基礎長さ(主開口方位側)(m)  
$L_{base,entrance,OS}$: 玄関の土間床等の周辺部の基礎長さ(その他の方位側)(m)  

In [26]:
for h in hs:
    h.a_ef_entrance = h.l_base_entrance_ms * h.l_base_entrance_os

In [27]:
show_result(lambda h: h.a_ef_entrance)

床断熱住宅 : 2.4843
基礎断熱住宅 : 2.4843


## 浴室の土間床等の面積

$ A_{EF,bath} = L_{base,bath,MS} \times L_{base,bath,OS} $

$A_{EF,bath}$: 浴室の土間床等の面積(m<sup>2</sup>)  
$L_{base,bath,MS}$: 浴室の土間床等の周辺部の基礎長さ(主開口方位側)(m)  
$L_{base,bath,OS}$: 浴室の土間床等の周辺部の基礎長さ(その他の方位側)(m)  

In [28]:
for h in hs:
    h.a_ef_bath = h.l_base_bath_ms * h.l_base_bath_os

In [29]:
show_result(lambda h: h.a_ef_bath)

床断熱住宅 : 3.3124000000000002
基礎断熱住宅 : 3.3124000000000002


## その他の土間床等の面積

$ A_{EF,other} = A_{F,1} - A_{EF,entrance} - A_{EF,bath} $

$ A_{EF,other} $: 土間床等の面積(玄関・浴室除く)(m<sup>2</sup>)  

ただし、「床断熱住宅」の場合は、0.0(m<sup>2</sup>)とする。

In [30]:
for h in hs:
    if h.floor_ins == FloorIns.FLOOR:
        h.a_ef_other = 0.0
    elif h.floor_ins == FloorIns.BASE:
        h.a_ef_other = h.a_f[0] - h.a_ef_entrance - h.a_ef_bath
    else:
        raise ValueError()

In [31]:
show_result(lambda h: h.a_ef_other)

床断熱住宅 : 0.0
基礎断熱住宅 : 45.05075762711864


## 床断熱した床の面積

$ A_{IF} = A_{F,1} - A_{EF,entrane} - A_{EF,bath} - A_{EF,other} - A_{IF,bath} $ 

$A_{IF}$: 床断熱した床の面積(m<sup>2</sup>)

In [32]:
for h in hs:
    h.a_if = h.a_f[0] - h.a_ef_entrance - h.a_ef_bath - h.a_ef_other

In [33]:
show_result(lambda h: h.a_if)

床断熱住宅 : 45.05075762711864
基礎断熱住宅 : 0.0


## 屋根又は天井

天井・屋根(下屋を含む)の面積は、オーバーハング床を想定しない限り、1階の床面積(土間床含む)と等しい。

$ A_{roof} = A_{F,1}  $ 

$A_{roof}$: 天井・屋根(下屋を含む)の面積 (m<sup>2</sup>)

In [34]:
for h in hs:
    h.a_roof = h.a_f[0]

In [35]:
show_result(lambda h: h.a_roof)

床断熱住宅 : 50.847457627118644
基礎断熱住宅 : 50.847457627118644


## 玄関等の土間床等の外周部の長さ（2017.7～）

2017年7月から。それまでは玄関とその他の基礎の区別がなかった。

外気側：  
南西: $L_{base,entrance,SW}=0.0$  
北西: $L_{base,entrance,NW}=L_{base,entrance,OS}$  
北東: $L_{base,entrance,NE}=L_{base,entrance,MS}$  
南東: $L_{base,entrance,SE}=0.0$  

床下側:  
$
L_{base,entrance,inside} = \left \{
\begin{array}{ll}
    L_{base,entrance,MS} + L_{base,entrance,OS} & \text{ ( floor insulted house ) } \\
    0.0 & \text{ ( insulated house ) }
\end{array}
\right.
$

In [36]:
for h in hs:
    h.l_base_entrance_outside = (0.0, h.l_base_entrance_os, h.l_base_entrance_ms, 0.0)
    h.l_base_entrance_inside = {
        FloorIns.FLOOR: h.l_base_entrance_ms + h.l_base_entrance_os,
        FloorIns.BASE: 0.0
    }[h.floor_ins]

In [37]:
show_result(lambda h: h.l_base_entrance_outside)

床断熱住宅 : (0.0, 1.82, 1.365, 0.0)
基礎断熱住宅 : (0.0, 1.82, 1.365, 0.0)


In [38]:
show_result(lambda h: h.l_base_entrance_inside)

床断熱住宅 : 3.185
基礎断熱住宅 : 0.0


## 浴室の土間床等の外周部の長さ

外気側：  
南西:  $L_{base,bath,SW}=0.0$  
北西:  $L_{base,bath,NW}=L_{base,bath,OS}$  
北東:  $L_{base,bath,NE}=L_{base,bath,MS}$  
南東:  $L_{base,bath,SE}=0.0$

室内側:  
$ L_{base,bath,inside} = L_{base,bath,MS} + L_{base,bath,OS} $

ただし、「床断熱住宅（外皮に接する浴室床なし）」、「床断熱住宅（浴室床断熱）」及び「基礎断熱住宅」の場合はいずれの長さも0mとする。

In [39]:
for h in hs:
    h.l_base_bath_outside = 0.0, h.l_base_bath_os, h.l_base_bath_ms, 0.0
    if h.floor_ins == FloorIns.FLOOR:
        h.l_base_bath_inside =  h.l_base_bath_ms + h.l_base_bath_os
    elif h.floor_ins == FloorIns.BASE:
        h.l_base_bath_inside =  0.0
    else:
        raise ValueError

In [40]:
show_result(lambda h: h.l_base_bath_outside)

床断熱住宅 : (0.0, 1.82, 1.82, 0.0)
基礎断熱住宅 : (0.0, 1.82, 1.82, 0.0)


In [41]:
show_result(lambda h: h.l_base_bath_inside)

床断熱住宅 : 3.64
基礎断熱住宅 : 0.0


## その他の土間床等の外周部の長さ（2018.4～）

南西: $L_{base,SW} = \left \{
\begin{array}{ll}
    0.0 & \text{ ( floor insulated house ) } \\
    L_{LS,1} - L_{base,entrance,SW} - L_{base,bath,SW} & \text{ ( base insulated house ) }
\end{array}
\right.
$

北西: $L_{base,NW} = \left \{
\begin{array}{ll}
    0.0 & \text{ ( floor insulated house ) } \\
    L_{SS,1} - L_{base,entrance,NW} - L_{base,bath,NW} & \text{ ( base insulated house ) }
\end{array}
\right.
$

北東: $L_{base,NE} = \left \{
\begin{array}{ll}
    0.0 & \text{ ( floor insulated house ) } \\
    L_{LS,1} - L_{base,entrance,NE} - L_{base,bath,NE} & \text{ ( base insulated house ) }
\end{array}
\right.
$

南東: $L_{base,SE} = \left \{
\begin{array}{ll}
    0.0 & \text{ ( floor insulated house ) } \\
    L_{SS,1} - L_{base,entrance,SE} - L_{base,bath,SE} & \text{ ( base insulated house ) }
\end{array}
\right.
$

室内側:  
$ L_{base,inside} = 0.0 $

In [42]:
for h in hs:

    if h.floor_ins == FloorIns.FLOOR:
        h.l_base_other_outside = (0.0, 0.0, 0.0, 0.0)
    elif h.floor_ins == FloorIns.BASE:
        h.l_base_other_outside = (
            h.l_ls[0] - h.l_base_entrance_outside[0] - h.l_base_bath_outside[0],
            h.l_ss[0] - h.l_base_entrance_outside[1] - h.l_base_bath_outside[1],
            h.l_ls[0] - h.l_base_entrance_outside[2] - h.l_base_bath_outside[2],
            h.l_ss[0] - h.l_base_entrance_outside[3] - h.l_base_bath_outside[3]
        )
    else:
        raise ValueError()    
    h.l_base_other_inside = 0.0

In [43]:
show_result(lambda h: h.l_base_other_outside)

床断熱住宅 : (0.0, 0.0, 0.0, 0.0)
基礎断熱住宅 : (10.609982280729707, 1.1524168280158131, 7.424982280729706, 4.7924168280158135)


In [44]:
show_result(lambda h: h.l_base_other_inside)

床断熱住宅 : 0.0
基礎断熱住宅 : 0.0


## 土間床周長(合計)

南西:   $ L_{prm,SW} = L_{base,SW} + L_{base,entrance,SW} + L_{base,bath,SW} $  
北西:   $ L_{prm,NW} = L_{base,NW} + L_{base,entrance,NW} + L_{base,bath,NW} $  
北東:   $ L_{prm,NE} = L_{base,NE} + L_{base,entrance,NE} + L_{base,bath,NE} $  
南東:   $ L_{prm,SE} = L_{base,SE} + L_{base,entrance,SE} + L_{base,bath,SE} $  

室内側: $ L_{prm,inside} = L_{base,inside} + L_{base,entrance,inside} + L_{base,bath,inside} $

In [45]:
for h in hs:
    h.l_prm_outside = (
        h.l_base_other_outside[0] + h.l_base_entrance_outside[0] + h.l_base_bath_outside[0],
        h.l_base_other_outside[1] + h.l_base_entrance_outside[1] + h.l_base_bath_outside[1],
        h.l_base_other_outside[2] + h.l_base_entrance_outside[2] + h.l_base_bath_outside[2],
        h.l_base_other_outside[3] + h.l_base_entrance_outside[3] + h.l_base_bath_outside[3]
    )
    h.l_prm_inside = h.l_base_other_inside + h.l_base_entrance_inside + h.l_base_bath_inside

In [46]:
show_result(lambda h: h.l_prm_outside)

床断熱住宅 : (0.0, 3.64, 3.185, 0.0)
基礎断熱住宅 : (10.609982280729707, 4.7924168280158135, 10.609982280729707, 4.7924168280158135)


In [47]:
show_result(lambda h: h.l_prm_inside)

床断熱住宅 : 6.825
基礎断熱住宅 : 0.0


## 玄関等を除く土間床等の外周部の長さ(m)（2017.7～2018.4）

2017.07まで「土間床周長の合計」のみを表示していたが、2017.07より、「土間床周長(玄関)」と「土間床周長(玄関除く)」に分けて記載することになった。  
簡易法の表記上、基礎の面積は、「玄関」と「（玄関以外の）その他」としてまとめる。  
本来であれば、「土間床周長(玄関除く)」は、「土間床周長(その他)」と「土間床周長(浴室)」の合計とすべきところである。しかし、「土間床周長(玄関)」と「土間床周長(浴室)」、「土間床周長(その他)」をそれぞれ小数点第二位まで四捨五入して合計した値は、「土間床周長の合計」を小数点第二位まで四捨五入した値に一致しない。そこで、「土間床周長の合計」を小数点第二位まで四捨五入した値から、「土間床周長(玄関)」を小数点第二位まで四捨五入した値を減じることにより、従来評価に変更がでないように対処する。

## 玄関等の基礎の面積

南西:  $ A_{base,entrance,SW} = L_{base,entrance,SW} \times h_{base,entrance} $  
北西:  $ A_{base,entrance,NW} = L_{base,entrance,NW} \times h_{base,entrance} $  
北東:  $ A_{base,entrance,NE} = L_{base,entrance,NE} \times h_{base,entrance} $  
南東:  $ L_{base,entrance,SE} = L_{base,entrance,SE} \times h_{base,entrance} $  

$A_{base,entrance,SW}$: 基礎(玄関)(南西)の面積(m<sup>2</sup>)  
$A_{base,entrance,NW}$: 基礎(玄関)(北西)の面積(m<sup>2</sup>)  
$A_{base,entrance,NE}$: 基礎(玄関)(北東)の面積(m<sup>2</sup>)  
$A_{base,entrance,SE}$: 基礎(玄関)(南東)の面積(m<sup>2</sup>)  

室内側:  $ L_{base,entrance,inside} = L_{base,entrance,inside} \times h_{base,entrance} $

$A_{base,entrance,inside}$: 基礎(玄関)(室内側)の面積(m<sup>2</sup>)  

In [48]:
for h in hs:
    h.a_base_entrance_outside = (
        h.l_base_entrance_outside[0] * h.h_base_entrance,
        h.l_base_entrance_outside[1] * h.h_base_entrance,
        h.l_base_entrance_outside[2] * h.h_base_entrance,
        h.l_base_entrance_outside[3] * h.h_base_entrance
    )    
    h.a_base_entrance_inside = h.l_base_entrance_inside * h.h_base_entrance

In [49]:
show_result(lambda h: h.a_base_entrance_outside)

床断熱住宅 : (0.0, 0.3276, 0.2457, 0.0)
基礎断熱住宅 : (0.0, 0.3276, 0.2457, 0.0)


In [50]:
show_result(lambda h: h.a_base_entrance_inside)

床断熱住宅 : 0.5733
基礎断熱住宅 : 0.0


## 浴室の基礎の面積

南西:  $ A_{base,bath,SW} = L_{base,bath,SW} \times h_{base,bath} $  
北西:  $ A_{base,bath,NW} = L_{base,bath,NW} \times h_{base,bath} $  
北東:  $ A_{base,bath,NE} = L_{base,bath,NE} \times h_{base,bath} $  
南東:  $ L_{base,bath,SE} = L_{base,bath,SE} \times h_{base,bath} $  

$A_{base,bath,SW}$: 基礎(浴室)(南西)の面積(m<sup>2</sup>)  
$A_{base,bath,NW}$: 基礎(浴室)(北西)の面積(m<sup>2</sup>)  
$A_{base,bath,NE}$: 基礎(浴室)(北東)の面積(m<sup>2</sup>)  
$A_{base,bath,SE}$: 基礎(浴室)(南東)の面積(m<sup>2</sup>)  

室内側:  $ L_{base,bath,inside} = L_{base,bath,inside} \times h_{base,bath} $

$A_{base,bath,inside}$: 基礎(浴室)(室内側)の面積(m<sup>2</sup>)  

In [51]:
for h in hs:
    h.a_base_bath_outside = (
        h.l_base_bath_outside[0] * h.h_base_bath,
        h.l_base_bath_outside[1] * h.h_base_bath,
        h.l_base_bath_outside[2] * h.h_base_bath,
        h.l_base_bath_outside[3] * h.h_base_bath
    )
    h.a_base_bath_inside = h.l_base_bath_inside * h.h_base_bath

In [52]:
show_result(lambda h: h.a_base_bath_outside)

床断熱住宅 : (0.0, 0.91, 0.91, 0.0)
基礎断熱住宅 : (0.0, 0.91, 0.91, 0.0)


In [53]:
show_result(lambda h: h.a_base_bath_inside)

床断熱住宅 : 1.82
基礎断熱住宅 : 0.0


## その他の基礎の面積

南西: $ A_{base,other,SW} = L_{base,SW} \times h_{base} $  
北西: $ A_{base,other,NW} = L_{base,NW} \times h_{base} $  
北東: $ A_{base,other,NE} = L_{base,NE} \times h_{base} $  
南東: $ A_{base,other,SE} = L_{base,SE} \times h_{base} $  

$A_{base,other,SW}$: 基礎断熱部分(玄関・浴室除く)(南西)の面積(m<sup>2</sup>)  
$A_{base,other,NW}$: 基礎断熱部分(玄関・浴室除く)(北西)の面積(m<sup>2</sup>)  
$A_{base,other,NE}$: 基礎断熱部分(玄関・浴室除く)(北東)の面積(m<sup>2</sup>)  
$A_{base,other,SE}$: 基礎断熱部分(玄関・浴室除く)(南東)の面積(m<sup>2</sup>)  

室内側:  $ A_{base,other,inside} = 0.0 $

$A_{base,other,inside}$: 基礎断熱部分(玄関・浴室除く)(室内側)の面積(m<sup>2</sup>)  

In [54]:
for h in hs:
    h.a_base_other_outside = (
        h.l_base_other_outside[0] * h.h_base,
        h.l_base_other_outside[1] * h.h_base,
        h.l_base_other_outside[2] * h.h_base,
        h.l_base_other_outside[3] * h.h_base
    )
    h.a_base_other_inside = 0.0

In [55]:
show_result(lambda h: h.a_base_other_outside)

床断熱住宅 : (0.0, 0.0, 0.0, 0.0)
基礎断熱住宅 : (5.304991140364853, 0.5762084140079066, 3.712491140364853, 2.3962084140079067)


In [56]:
show_result(lambda h: h.a_base_other_inside)

床断熱住宅 : 0.0
基礎断熱住宅 : 0.0


## 基礎断熱部分の面積の合計

南西:  $ A_{prm,SW} = A_{base,SW} + A_{base,entrance,SW} + A_{base,b,SW} $  
北西:  $ A_{prm,NW} = A_{base,NW} + A_{base,entrance,NW} + A_{base,b,NW} $  
北東:  $ A_{prm,NE} = A_{base,NE} + A_{base,entrance,NE} + A_{base,b,NE} $  
南東:  $ A_{prm,SE} = A_{base,SE} + A_{base,entrance,SE} + A_{base,b,SE} $  

$A_{prm,SW}$: 基礎断熱部分(南西)の面積の合計(m<sup>2</sup>)  
$A_{prm,NW}$: 基礎断熱部分(北西)の面積の合計(m<sup>2</sup>)  
$A_{prm,NE}$: 基礎断熱部分(北東)の面積の合計(m<sup>2</sup>)  
$A_{prm,SE}$: 基礎断熱部分(南東)の面積の合計(m<sup>2</sup>)  

室内側:  $ A_{prm,IS} = A_{base,IS} + A_{base,entrance,IS} + A_{base,b,IS} $

$A_{prm,IS}$: 基礎断熱部分(室内側)の面積の合計(m<sup>2</sup>)  

In [57]:
for h in hs:
    h.a_base_total_outside = (
        h.a_base_other_outside[0] + h.a_base_entrance_outside[0] + h.a_base_bath_outside[0],
        h.a_base_other_outside[1] + h.a_base_entrance_outside[1] + h.a_base_bath_outside[1],
        h.a_base_other_outside[2] + h.a_base_entrance_outside[2] + h.a_base_bath_outside[2],
        h.a_base_other_outside[3] + h.a_base_entrance_outside[3] + h.a_base_bath_outside[3]
    )
    h.a_base_total_inside = h.a_base_other_inside + h.a_base_entrance_inside + h.a_base_bath_inside

In [58]:
show_result(lambda h: h.a_base_total_outside)

床断熱住宅 : (0.0, 1.2376, 1.1557, 0.0)
基礎断熱住宅 : (5.304991140364853, 1.8138084140079065, 4.868191140364853, 2.3962084140079067)


In [59]:
show_result(lambda h: h.a_base_total_inside)

床断熱住宅 : 2.3933
基礎断熱住宅 : 0.0


## 玄関等を除く土間床等の外周部の長さ(m)（2017.7～2018.4）

2017.07まで「土間床周長の合計」のみを表示していたが、2017.07より、「土間床周長(玄関)」と「土間床周長(玄関除く)」に分けて記載することになった。  
簡易法の表記上、基礎の面積は、「玄関」と「（玄関以外の）その他」としてまとめる。  
本来であれば、「土間床周長(玄関除く)」は、「土間床周長(その他)」と「土間床周長(浴室)」の合計とすべきところである。しかし、「土間床周長(玄関)」と「土間床周長(浴室)」、「土間床周長(その他)」をそれぞれ小数点第二位まで四捨五入して合計した値は、「土間床周長の合計」を小数点第二位まで四捨五入した値に一致しない。そこで、「土間床周長の合計」を小数点第二位まで四捨五入した値から、「土間床周長(玄関)」を小数点第二位まで四捨五入した値を減じることにより、従来評価に変更がでないように対処する。

## 壁面積(開口部含む)

1階:  
南西: $ A_{srf,1,SW} = L_{LS,1} \times h_1 $  
北西: $ A_{srf,1,NW} = L_{SS,1} \times h_1 $  
北東: $ A_{srf,1,NE} = L_{LS,1} \times h_1 $  
北西: $ A_{srf,1,SE} = L_{SS,1} \times h_1 $

2階:  
南西: $ A_{srf,2,SW} = L_{LS,2} \times h_2 $  
北西: $ A_{srf,2,NW} = L_{SS,2} \times h_2 $  
北東: $ A_{srf,2,NE} = L_{LS,2} \times h_2 $  
北西: $ A_{srf,2,SE} = L_{SS,2} \times h_2 $

In [60]:
for h in hs:
    h.a_srf = [
        (h.l_ls[0] * h.h[0], h.l_ss[0] * h.h[0], h.l_ls[0] * h.h[0], h.l_ss[0] * h.h[0]),
        (h.l_ls[1] * h.h[1], h.l_ss[1] * h.h[1], h.l_ls[1] * h.h[1], h.l_ss[1] * h.h[1])
    ]

In [61]:
show_result(lambda h: h.a_srf)

床断熱住宅 : [(30.768948614116148, 13.898008801245858, 30.768948614116148, 13.898008801245858), (22.396232256298443, 12.7441986952085, 22.396232256298443, 12.7441986952085)]
基礎断熱住宅 : [(30.768948614116148, 13.898008801245858, 30.768948614116148, 13.898008801245858), (22.396232256298443, 12.7441986952085, 22.396232256298443, 12.7441986952085)]


## 外皮面積の合計(基礎の面積を除く)

$ A'_{env} = A_{F,1} + A_{roof} + A_{srf,1,SW} + A_{srf,1,NW} + A_{srf,1,NE} + A_{srf,1,SE} + A_{srf,2,SW} + A_{srf,2,NW} + A_{srf,2,NE} + A_{srf,2,SE} $

$ A'_{env}$: 外皮面積の合計(基礎の面積を除く)(m<sup>2</sup>)

In [62]:
for h in hs:
    h.a_dash_env = h.a_f[0] + h.a_roof + sum(h.a_srf[0]) + sum(h.a_srf[1])

In [63]:
show_result(lambda h: h.a_dash_env)

床断熱住宅 : 261.30969198797516
基礎断熱住宅 : 261.30969198797516


## 外皮面積の合計(m<sup>2</sup>)

$ A_{env} = A'_{env} + A_{prm,SW} + A_{prm,NW} + A_{prm,NE} + A_{prm,SE} + A_{prm,IS} $

$ A_{env} $: 外皮面積の合計(m<sup>2</sup>)

In [64]:
for h in hs:
    h.a_env = h.a_dash_env + sum(h.a_base_total_outside) + h.a_base_total_inside

In [65]:
show_result(lambda h: h.a_env)

床断熱住宅 : 266.0962919879752
基礎断熱住宅 : 275.6928910967207


## 開口部の面積の合計

$ A_{opn} = A'_{env} \times r_{opn} $

$A_{opn}$: 開口部の面積の合計(m<sup>2</sup>)

In [66]:
for h in hs:
    h.a_opn = h.a_dash_env * h.r_opn

In [67]:
show_result(lambda h: h.a_opn)

床断熱住宅 : 36.58335687831652
基礎断熱住宅 : 36.58335687831652


## 窓の面積の合計

$ A_{wnd,total} = A_{opn} - A_{door,entrance} - A_{door,backentrance} $

$A_{wnd,total}$: 窓の面積(m<sup>2</sup>)

In [68]:
for h in hs:
    h.a_wnd_total = h.a_opn - h.a_door_entrance - h.a_door_backentrance

In [69]:
show_result(lambda h: h.a_wnd_total)

床断熱住宅 : 33.073356878316524
基礎断熱住宅 : 33.073356878316524


## 窓の面積

南西側: $ A_{wnd,SW} = A_{wnd} \times r_{wnd,SW} $  
北西側: $ A_{wnd,NW} = A_{wnd} \times r_{wnd,NW} $  
北東側: $ A_{wnd,NE} = A_{wnd} \times r_{wnd,NE} $  
南東側: $ A_{wnd,SE} = A_{wnd} \times r_{wnd,SE} $  

$ A_{wnd,SW}$: 窓の面積(南西)(m<sup>2</sup>)  
$ A_{wnd,NW}$: 窓の面積(北西)(m<sup>2</sup>)  
$ A_{wnd,NE}$: 窓の面積(北東)(m<sup>2</sup>)  
$ A_{wnd,SE}$: 窓の面積(南東)(m<sup>2</sup>)  

In [70]:
for h in hs:
    h.a_wnd = h.a_wnd_total * h.r_wnd[0], h.a_wnd_total * h.r_wnd[1], h.a_wnd_total * h.r_wnd[2], h.a_wnd_total * h.r_wnd[3]

In [71]:
show_result(lambda h: h.a_wnd)

床断熱住宅 : (22.6949374899008, 2.3845890309266213, 3.628147249551323, 4.365683107937781)
基礎断熱住宅 : (22.6949374899008, 2.3845890309266213, 3.628147249551323, 4.365683107937781)


## ドアの面積

南西側: $ A_{door,SW} = 0.0 $  
北西側: $ A_{door,NW} = A_{door,entrance} $   
北東側: $ A_{door,NE} = A_{door,backentrance} $  
南東側: $ A_{door,SE} = 0.0 $  

$A_{door}$: ドアの面積(m<sup>2</sup>)

In [72]:
for h in hs:
    h.a_door = 0.0, h.a_door_entrance, h.a_door_backentrance, 0.0

In [73]:
show_result(lambda h: h.a_door_entrance)

床断熱住宅 : 1.89
基礎断熱住宅 : 1.89


## 壁の面積

南西側: $ A_{wall,SW} = A_{srf,1,SW} + A_{srf,2,SW} - A_{wnd,SW} - A_{door,SW} $  
北西側: $ A_{wall,NW} = A_{srf,1,NW} + A_{srf,2,NW} - A_{wnd,NW} - A_{door,NW} $  
北東側: $ A_{wall,NE} = A_{srf,1,NE} + A_{srf,2,NE} - A_{wnd,NE} - A_{door,NE} $  
南東側: $ A_{wall,SE} = A_{srf,1,SE} + A_{srf,2,SE} - A_{wnd,SE} - A_{door,SE} $  

$ A_{wall}$: 壁の面積(m<sup>2</sup>)

In [74]:
for h in hs:
    h.a_wall = (
        h.a_srf[0][0] + h.a_srf[1][0] - h.a_wnd[0] - h.a_door[0],
        h.a_srf[0][1] + h.a_srf[1][1] - h.a_wnd[1] - h.a_door[1],
        h.a_srf[0][2] + h.a_srf[1][2] - h.a_wnd[2] - h.a_door[2],
        h.a_srf[0][3] + h.a_srf[1][3] - h.a_wnd[3] - h.a_door[3]
    )

In [75]:
show_result(lambda h: h.a_wall)

床断熱住宅 : (30.47024338051379, 22.367618465527734, 47.917033620863265, 22.276524388516577)
基礎断熱住宅 : (30.47024338051379, 22.367618465527734, 47.917033620863265, 22.276524388516577)


# 数字の丸め

## 外皮の部位の面積の合計

In [76]:
for h, doc in zip(hs, docs):
    doc['外皮の部位の面積の合計'] = round2(h.a_env)

## 床面積の合計

In [77]:
for h, doc in zip(hs, docs):
    doc['床面積の合計'] = round2(h.a_f_total)

## 屋根又は天井の面積

In [78]:
for h, doc in zip(hs, docs):
    doc['屋根又は天井の面積'] = round2(h.a_roof)

## 壁の面積 

In [79]:
for h, doc in zip(hs, docs):
    doc['南西に面した壁の面積'] = round2(h.a_wall[0])
    doc['北西に面した壁の面積'] = round2(h.a_wall[1])
    doc['北東に面した壁の面積'] = round2(h.a_wall[2])
    doc['南東に面した壁の面積'] = round2(h.a_wall[3])

## ドアの面積

In [80]:
for h, doc in zip(hs, docs):
    doc['南西に面したドアの面積'] = round2(h.a_door[0])
    doc['北西に面したドアの面積'] = round2(h.a_door[1])
    doc['北東に面したドアの面積'] = round2(h.a_door[2])
    doc['南東に面したドアの面積'] = round2(h.a_door[3])

## 窓の面積

In [81]:
for h, doc in zip(hs, docs):
    doc['南西に面した窓の面積'] = round2(h.a_wnd[0])
    doc['北西に面した窓の面積'] = round2(h.a_wnd[1])
    doc['北東に面した窓の面積'] = round2(h.a_wnd[2])
    doc['南東に面した窓の面積'] = round2(h.a_wnd[3])

## 床断熱した床の面積

In [82]:
for h, doc in zip(hs, docs):
    doc['床断熱した床の面積'] = round2(h.a_if)

## 玄関等を除く基礎の面積

2017.07まで「基礎の面積の合計」のみを表示していたが、2017.07より、「基礎の面積(玄関)」と「基礎の面積(玄関除く)」に分けて記載することになった。  
簡易法の表記上、基礎の面積は、「玄関」と「（玄関以外の）その他」としてまとめる。  
本来であれば、「基礎の面積(玄関除く)」は、「基礎の面積(その他)」と「基礎の面積(浴室)」の合計とすべきところである。しかし、「基礎の面積(玄関)」と「基礎の面積(浴室)」、「基礎の面積(その他)」をそれぞれ小数点第二位まで四捨五入して合計した値は、「基礎の面積の合計」を小数点第二位まで四捨五入した値に一致しない。そこで、「基礎の面積の合計」を小数点第二位まで四捨五入した値から、「基礎の面積(玄関)」を小数点第二位まで四捨五入した値を減じることにより、従来評価に変更がでないように対処する。

In [83]:
for h, doc in zip(hs, docs):
    doc['南西に面した玄関等を除く基礎の面積'] = round2(round2(h.a_base_total_outside[0]) - round2(h.a_base_entrance_outside[0]))
    doc['北西に面した玄関等を除く基礎の面積'] = round2(round2(h.a_base_total_outside[1]) - round2(h.a_base_entrance_outside[1]))
    doc['北東に面した玄関等を除く基礎の面積'] = round2(round2(h.a_base_total_outside[2]) - round2(h.a_base_entrance_outside[2]))
    doc['南東に面した玄関等を除く基礎の面積'] = round2(round2(h.a_base_total_outside[3]) - round2(h.a_base_entrance_outside[3]))
    doc['床下に面した玄関等を除く基礎の面積'] = round2(round2(h.a_base_total_inside) - round2(h.a_base_entrance_inside))

## 玄関等の基礎の面積

In [84]:
for h, doc in zip(hs, docs):
    doc['南西に面した玄関等の基礎の面積'] = round2(h.a_base_entrance_outside[0])
    doc['北西に面した玄関等の基礎の面積'] = round2(h.a_base_entrance_outside[1])
    doc['北東に面した玄関等の基礎の面積'] = round2(h.a_base_entrance_outside[2])
    doc['南東に面した玄関等の基礎の面積'] = round2(h.a_base_entrance_outside[3])
    doc['床下に面した玄関等の基礎の面積'] = round2(h.a_base_entrance_inside)

## 玄関等を除く土間床等の外周部の長さ

2017.07まで「土間床周長の合計」のみを表示していたが、2017.07より、「土間床周長(玄関)」と「土間床周長(玄関除く)」に分けて記載することになった。  
簡易法の表記上、基礎の面積は、「玄関」と「（玄関以外の）その他」としてまとめる。  
本来であれば、「土間床周長(玄関除く)」は、「土間床周長(その他)」と「土間床周長(浴室)」の合計とすべきところである。しかし、「土間床周長(玄関)」と「土間床周長(浴室)」、「土間床周長(その他)」をそれぞれ小数点第二位まで四捨五入して合計した値は、「土間床周長の合計」を小数点第二位まで四捨五入した値に一致しない。そこで、「土間床周長の合計」を小数点第二位まで四捨五入した値から、「土間床周長(玄関)」を小数点第二位まで四捨五入した値を減じることにより、従来評価に変更がでないように対処する。

In [85]:
for h, doc in zip(hs, docs):
    doc['南西に面した玄関等を除く土間床等の外周部の長さ'] = round2(round2(h.l_prm_outside[0]) - round2(h.l_base_entrance_outside[0]))
    doc['北西に面した玄関等を除く土間床等の外周部の長さ'] = round2(round2(h.l_prm_outside[1]) - round2(h.l_base_entrance_outside[1]))
    doc['北東に面した玄関等を除く土間床等の外周部の長さ'] = round2(round2(h.l_prm_outside[2]) - round2(h.l_base_entrance_outside[2]))
    doc['南東に面した玄関等を除く土間床等の外周部の長さ'] = round2(round2(h.l_prm_outside[3]) - round2(h.l_base_entrance_outside[3]))
    doc['床下に面した玄関等を除く土間床等の外周部の長さ'] = round2(round2(h.l_prm_inside) - round2(h.l_base_entrance_inside))

## 玄関等の土間床等の外周部の長さ

In [86]:
for h, doc in zip(hs, docs):
    doc['南西に面した玄関等の土間床等の外周部の長さ'] = round2(h.l_base_entrance_outside[0])
    doc['北西に面した玄関等の土間床等の外周部の長さ'] = round2(h.l_base_entrance_outside[1])
    doc['北東に面した玄関等の土間床等の外周部の長さ'] = round2(h.l_base_entrance_outside[2])
    doc['南東に面した玄関等の土間床等の外周部の長さ'] = round2(h.l_base_entrance_outside[3])
    doc['床下に面した玄関等の土間床等の外周部の長さ'] = round2(h.l_base_entrance_inside)
    print((doc['南西に面した玄関等の土間床等の外周部の長さ'],
           doc['北西に面した玄関等の土間床等の外周部の長さ'],
           doc['北東に面した玄関等の土間床等の外周部の長さ'],
           doc['南東に面した玄関等の土間床等の外周部の長さ'],
           doc['床下に面した玄関等の土間床等の外周部の長さ']))

(0.0, 1.82, 1.37, 0.0, 3.19)
(0.0, 1.82, 1.37, 0.0, 0.0)


# テスト

In [87]:
wb = xlrd.open_workbook('publicated_value.xlsx')
sheet = wb.sheet_by_name('test2017')
values = [sheet.row_values(row) for row in range(sheet.nrows)]

In [88]:
for i, value in enumerate(values):
    if i != 0:
        print(value[0])
        is_ok = True
        for j, doc in enumerate(docs):
            real = doc[value[0]]
            expected = value[j+1]
            if real != expected:
                print('house ' + str(j) + ' error: real = ' + str(real) + ' expected = ' + str(expected) )
                is_ok = False
        if is_ok == True:
            print('OK')


外皮の部位の面積の合計
OK
床面積の合計
OK
屋根又は天井の面積
OK
南西に面した壁の面積
OK
北西に面した壁の面積
OK
北東に面した壁の面積
OK
南東に面した壁の面積
OK
南西に面したドアの面積
OK
北西に面したドアの面積
OK
北東に面したドアの面積
OK
南東に面したドアの面積
OK
南西に面した窓の面積
OK
北西に面した窓の面積
OK
北東に面した窓の面積
OK
南東に面した窓の面積
OK
床断熱した床の面積
OK
南西に面した玄関等を除く基礎の面積
OK
北西に面した玄関等を除く基礎の面積
OK
北東に面した玄関等を除く基礎の面積
OK
南東に面した玄関等を除く基礎の面積
OK
床下に面した玄関等を除く基礎の面積
OK
南西に面した玄関等の基礎の面積
OK
北西に面した玄関等の基礎の面積
OK
北東に面した玄関等の基礎の面積
OK
南東に面した玄関等の基礎の面積
OK
床下に面した玄関等の基礎の面積
OK
南西に面した玄関等を除く土間床等の外周部の長さ
OK
北西に面した玄関等を除く土間床等の外周部の長さ
OK
北東に面した玄関等を除く土間床等の外周部の長さ
OK
南東に面した玄関等を除く土間床等の外周部の長さ
OK
床下に面した玄関等を除く土間床等の外周部の長さ
OK
南西に面した玄関等の土間床等の外周部の長さ
OK
北西に面した玄関等の土間床等の外周部の長さ
OK
北東に面した玄関等の土間床等の外周部の長さ
OK
南東に面した玄関等の土間床等の外周部の長さ
OK
床下に面した玄関等の土間床等の外周部の長さ
OK
